In [49]:
import warnings
warnings.filterwarnings('once')


In [50]:

import numpy as np
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Model, load_model
from tensorflow.keras.applications.vgg16 import VGG16, preprocess_input
from tensorflow.keras.preprocessing.image import ImageDataGenerator, img_to_array, load_img
from tensorflow.keras.optimizers import Adam
import os
from glob import glob

In [51]:
IMAGE_SIZE = [224, 224]
TRAIN_PATH = 'X-rays/train'
VALID_PATH = 'X-rays/test'
BATCH_SIZE = 32
EPOCHS = 1
MODEL_PATH = 'penumonia.h5'

In [52]:
def create_model():
    base_model = VGG16(input_shape=[224,224,3], weights='imagenet', include_top=False)
    for k in base_model.layers:
        k.trainable = False

    z = Flatten()(base_model.output)
    output_layer = Dense(len(glob(f'{TRAIN_PATH}/*')), activation='softmax')(z)

    model = Model(inputs=base_model.input, outputs=output_layer)
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

In [53]:
def get_data_generators():
    train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        rotation_range = 10,
        zoom_range=0.25,
        horizontal_flip=True,
        vertical_flip=False,

    )
    
    test_datagen = ImageDataGenerator(rescale=1./255)
    
    train_generator = train_datagen.flow_from_directory(
        TRAIN_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    validation_generator = test_datagen.flow_from_directory(
        VALID_PATH,
        target_size=IMAGE_SIZE,
        batch_size=BATCH_SIZE,
        class_mode='categorical'
    )
    
    return train_generator, validation_generator

In [54]:
def train_model(model, train_generator, validation_generator):
    
    model.fit(
        train_generator,
        epochs=EPOCHS,
        verbose=1,
        validation_data=validation_generator,
        steps_per_epoch=len(train_generator),
        validation_steps=len(validation_generator),

    )
    model.save(MODEL_PATH)


In [55]:
def predict_image(model, img_path):
    img = load_img(img_path, target_size=IMAGE_SIZE)
    z = img_to_array(img)
    z = np.expand_dims(z, axis=0)
    z = preprocess_input(z)

    prediction = model.predict(z)
    return prediction

In [56]:
model = create_model()
train_gen, val_gen = get_data_generators()
train_model(model, train_gen, val_gen)
model = load_model(MODEL_PATH)


Found 5216 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


/Users/prashanthkumargunda/anaconda3/lib/python3.11/site-packages/keras/src/preprocessing/image.py:2611: DeprecationWarning: Please import `affine_transform` from the `scipy.ndimage` namespace; the `scipy.ndimage.interpolation` namespace is deprecated and will be removed in SciPy 2.0.0.
  ndimage.interpolation.affine_transform(


163/163 [==============================] - 819s 5s/step - loss: 0.2333 - accuracy: 0.9139 - val_loss: 0.3376 - val_accuracy: 0.8894


/Users/prashanthkumargunda/anaconda3/lib/python3.11/site-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [57]:
img_path = 'X-rays/val/NORMAL/NORMAL2-IM-1427-0001.jpeg'
prediction = predict_image(model, img_path)
result = np.argmax(prediction)
if result == 0:
    print("Result is Normal")
else:
    print("Person is affected by Pneumonia")

1/1 [==============================] - 0s 192ms/step
Result is Normal


In [60]:
!streamlit run streamlit_app.py


  You can now view your Streamlit app in your browser.

  Local URL: http://localhost:8501
  Network URL: http://192.168.200.140:8501

1/1 [==============================] - 0s 156ms/step
^C
  Stopping...
